## ERA V7 

- All from V6 
- Added elevation
- Added total precipitation
- Used to create static images of hardiness zones

In [1]:
# # Import goes here 
import xarray as xr
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap, HeatMapWithTime, TimeSliderChoropleth
import geopandas as gpd
import numpy as np
from sklearn.cluster import DBSCAN
import shapely as shp
import requests
import contextily as ctx
import cfgrib
from matplotlib.colors import LinearSegmentedColormap, ListedColormap, BoundaryNorm
import matplotlib as mpl
import warnings
warnings.filterwarnings('ignore')

In [65]:
# Utils
def get_hardiness_zones(ds, boundary, temp_range=[-26, 40], precipitation_range=[0, 3000], months=list(range(1,13)), years=[1950, 2025], end_month="DEC", yearly_mean=True): 
    """
    Get filtered temperature data. 
    """
    # Filter Date
    ds_africa = ds.sel(time=slice(f"{years[0]}-01-01", f"{years[1]}-12-31"))
    ds_africa = ds_africa.where((precipitation_range[0] <= ds.tp) & (ds.tp <= precipitation_range[1]))

    # Filter Specific months 
    ds_africa = ds_africa.sel(time=ds_africa.time.dt.month.isin(months))

    # Get the yearly mean
    if yearly_mean:
        ds_africa = ds_africa.assign_coords(year=ds_africa.time.to_index().to_period(f"Y-{end_month}"))
        ds_africa = ds_africa.groupby("year").mean(dim="time", skipna=True)

    africa_df = ds_africa.to_dataframe().reset_index()
    africa_df["time"] = gpd.pd.to_datetime(africa_df["year"].astype(str), format="%Y")
 
    africa_df = africa_df[["time", "t2m", "tp", "latitude", "longitude"]]

    africa_df.dropna(inplace=True)
    africa_df.reset_index(drop=True, inplace=True)
    africa_df = africa_df.round(6)
    geometry = gpd.points_from_xy(africa_df.longitude, africa_df.latitude)
    africa_df = gpd.GeoDataFrame(africa_df, geometry=geometry, crs="EPSG:4326") # , crs="EPSG:4326"

    # Select points within the boundary
    geo_df = africa_df.loc[africa_df["geometry"].within(boundary)]
    print("Total points within boundary:", len(geo_df))
    
    # # Filter temperature range
    geo_df.loc[:, "zones"] = np.where(((temp_range[0] <= geo_df.t2m) & (geo_df.t2m <= temp_range[1])), 1, 0)
    geo_df.reset_index(drop=True, inplace=True)
    print("Total points within temperature range:", len(geo_df[geo_df["zones"] == 1]))

    return geo_df


def get_boundaries(boundary, countries): 
    """Give a set of boundaries filter and return a union of the geometries within the given countries."""

    boundary = boundary[boundary["ADMIN"].isin(countries)]["geometry"]  # "ADMIN"
    return boundary

# Clustering 
def cluster_temperature_data(df, eps=0.15, hull_ratio=0.3): 
    """Cluster points into an area to remove points within it."""
    hulls = []
    times = df["time"].dt.year.unique()
    for t in times: 
        #print(t)
        df_temp = df[df["time"].dt.year == t]
        coords = df_temp[["longitude", "latitude"]].values
        db = DBSCAN(eps=eps, min_samples=2).fit(coords)
        labels = db.labels_

        # Create convex hull
        for l in np.unique(labels):
            if l == -1: 
                continue
            cluster_points = coords[labels == l]
            multipoint = shp.MultiPoint(cluster_points)
            hull = shp.concave_hull(multipoint, ratio=hull_ratio, allow_holes=True)
            hulls.append({"time": t, "geometry": hull})

    return hulls


def split_dataframe(df, split_size):
    """
    Split a dataframe into smaller chunks for elevation API requests.
    """
    
    splits = []
    for start in range(0, len(df), split_size):
        end = start + split_size
        chunk = df.iloc[start:end].to_dict(orient="records")
        splits.append({"locations": chunk})
    return splits

### Load and filter temperature data for hardiness zones

In [38]:
# Read datasets
print("Loading ERA5 data...")
# ds = xr.load_dataset("./dataset/africa_t2m.grib", engine="cfgrib", decode_timedelta=True) # "./dataset/africa_t2m.grib"\
ds = cfgrib.open_datasets("./dataset/t2m_tp.grib")
ds_t2m ,ds_tp = ds[0], ds[2]
ds_t2m = ds_t2m - 273.15 # Convert from Kelvin to Celsius

# Convert precipitation from m to mm
# See ERA5 monthly averaged reanalysis https://confluence.ecmwf.int/pages/viewpage.action?pageId=197702790
ds_tp = ds_tp * 1000 * 30 * 12 # Convert to yearly mm 
ds_combined = xr.merge([ds_t2m, ds_tp])

Loading ERA5 data...


In [97]:
countries = ["Kenya","Ethiopia", "Uganda"] #, "United Republic of Tanzania"]
coffee_type = "robusta" # "arabica" or "robusta"
year = [1950, 2026]
months =  [1, 2, 3, 9, 10, 11, 12]  #Growing season # list(range(1,13)) 
end_month = "MAR"
arabica_hardiness = {
    "optimal_temp_range": [14, 28],
    "absolute_temp_range": [10, 34],
    "precipitation_range": [1500, 2000],
    "elevation_range": [800, 2000]
}

robusta_hardiness = {
    "optimal_temp_range": [20, 30],
    "absolute_temp_range": [12, 36],
    "precipitation_range": [900, 4000],
    "elevation_range": [0, 800]
}

hardiness = arabica_hardiness if coffee_type == "arabica" else robusta_hardiness

temp_range = hardiness["absolute_temp_range"]
elevation_range = hardiness["elevation_range"]
precipitation_range = hardiness["precipitation_range"]
# Growing season Done
# October to January Done
# Rearrange time timeframe Done
# remove urban areas and take 
# Add elevation layer Done

# Read shp file 
print("Loading shp file...")

boundaries = gpd.read_file("dataset/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")
boundary = get_boundaries(boundaries, countries)
boundary_union = boundary.union_all()
# Get temperature data
print("Filtering temperature data...")
temp_df = get_hardiness_zones(ds_combined, boundary_union,temp_range=temp_range, precipitation_range=precipitation_range, months=months, years=year, end_month=end_month)
temp_df = temp_df[temp_df["zones"] == 1] # Get filtered temperature data

# Read lakes and rivers
print("Loading lakes and rivers...")
lakes = gpd.read_file("./dataset/ne_10m_lakes/ne_10m_lakes.shp")
rivers = gpd.read_file("./dataset/ne_10m_rivers_lake_centerlines/ne_10m_rivers_lake_centerlines.shp")

lakes = lakes[lakes["geometry"].intersects(boundary_union)]
rivers = rivers[rivers["geometry"].intersects(boundary_union)]

temp_df = temp_df[~temp_df["geometry"].intersects(lakes.union_all())]
temp_df = temp_df[~temp_df["geometry"].intersects(rivers.union_all())]
temp_df.reset_index(drop=True, inplace=True)

# Get elevation from SRTM CGIAR Elevation Database
print("Getting elevation data...")  
url = "https://api.open-elevation.com/api/v1/lookup"
elevation_points = temp_df[["longitude", "latitude"]].drop_duplicates()

data_chunks = split_dataframe(elevation_points, split_size=2000)
print(f"Total elevation points and chunks: {len(elevation_points)}, {len(data_chunks)}")
elevation_list = []
for i, data in enumerate(data_chunks): # [data1, data2, data3, data4]
    print(f"getting elevation batch {i}...")
    res = requests.post(url, json=data)
    print(res.status_code, res.reason)
    elevation = res.json()["results"]
    elevation = gpd.GeoDataFrame(elevation)
    elevation = elevation.round(6)
    geometry = gpd.points_from_xy(elevation.longitude, elevation.latitude)
    elevation = elevation.set_geometry(geometry, crs=temp_df.crs)
    elevation_list.append(elevation)

print("Merging elevation data...")
elevation = gpd.pd.concat(elevation_list, ignore_index=True)
elevation.reset_index(drop=True, inplace=True)
temp_df = temp_df.merge(elevation[["elevation", "geometry"]], on="geometry", how="left")
temp_df = temp_df[temp_df["elevation"].between(elevation_range[0], elevation_range[1])]
temp_df.reset_index(drop=True, inplace=True)

# # temp_df.to_file("filtered_temp_data.geojson", driver="GeoJSON")

Loading shp file...
Filtering temperature data...
Total points within boundary: 259902
Total points within temperature range: 257566
Loading lakes and rivers...
Getting elevation data...
Total elevation points and chunks: 15151, 8
getting elevation batch 0...
200 OK
getting elevation batch 1...
200 OK
getting elevation batch 2...
200 OK
getting elevation batch 3...
200 OK
getting elevation batch 4...
200 OK
getting elevation batch 5...
200 OK
getting elevation batch 6...
200 OK
getting elevation batch 7...
200 OK
Merging elevation data...


In [100]:
# Cluster hardiness zones
clustered_hardiness_zones = cluster_temperature_data(temp_df, eps=0.115, hull_ratio=0.02)
clustered_hardiness_zones = gpd.GeoDataFrame(clustered_hardiness_zones, crs=temp_df.crs)
clustered_hardiness_zones["time"] = gpd.pd.to_datetime(clustered_hardiness_zones["time"], format="%Y")

In [ ]:
# Get area of hardiness zones
yearly_hardiness_area = clustered_hardiness_zones.dissolve(by="time", aggfunc="sum")
yearly_hardiness_area = yearly_hardiness_area.to_crs({'proj':'cea'}).area / 10**6
# In km^2
# yearly_hardiness_area
yearly_hardiness_area.plot(title="Area of Hardiness Zones Over the Years", ylabel="Area (km^2)", xlabel="Year")

In [102]:
yearly_hardiness_area = yearly_hardiness_area.reset_index()
yearly_hardiness_area["file_name"] = [f"hardiness_zones_{year}.jpeg" for year in yearly_hardiness_area["time"].values]
yearly_hardiness_area.rename(columns={0: "area_km2"}, inplace=True)
yearly_hardiness_area.to_csv(f"./results/{coffee_type}/hardiness_zones.csv", index=False)

In [ ]:
n_sample = 1000 

years = temp_df["time"].dt.year.unique()
print(years)
print(coffee_type)
for year in years:
    print(year)
    yearly_temp_df = temp_df[temp_df["time"].dt.year == year]
    yearly_temp_df.to_crs(epsg=3857, inplace=True)
    yearly_clustered = clustered_hardiness_zones[clustered_hardiness_zones["time"].dt.year == year]
    yearly_clustered.to_crs(epsg=3857, inplace=True)

    # Create custom colormap
    ranges = [min(yearly_temp_df["t2m"].min(), hardiness["optimal_temp_range"][0]), 
          hardiness["optimal_temp_range"][0], 
          hardiness["optimal_temp_range"][1],
          max(yearly_temp_df["t2m"].max(), hardiness["optimal_temp_range"][1])]

    ranges_n = [(v - ranges[0]) / (ranges[-1] - ranges[0]) for v in ranges]
    ranges_n_diff = [max(round((j - i) * n_sample), 1) for i, j in zip(ranges_n[:-1], ranges_n[1:])]
    #print(ranges_n_diff, ranges_n, ranges)
    # Build custom cmap
    left_green = mpl.colormaps['Greens'].resampled(ranges_n_diff[0])
    left_green = left_green(np.linspace(0, 1, ranges_n_diff[0]))

    mid_green = np.array([left_green[-1, :]] * ranges_n_diff[1])

    right_green = mpl.colormaps['Greens_r'].resampled(ranges_n_diff[2])
    right_green = right_green(np.linspace(0, 1, ranges_n_diff[2]))

    newGreen = np.vstack((left_green, mid_green, right_green))
    customGreen = ListedColormap(newGreen, name='CustomGreen')


    # Plot data 
    fig, ax = plt.subplots(figsize=(15, 15), dpi=200)
    yearly_temp_df.plot(column="t2m",ax=ax, markersize=15, marker="s", cmap=customGreen, alpha=0.6, label="Temperature Points", legend=True, legend_kwds={'label': "temperature", 'shrink': 0.5})
    #yearly_clustered.plot(ax=ax, markersize=10, color="green", alpha=0.4, label="Hardiness Zones")
    rivers.to_crs(epsg=3857).plot(ax=ax, color="blue", alpha=0.5, label="Water Bodies")
    lakes.to_crs(epsg=3857).plot(ax=ax, color="blue", alpha=0.5, label="Water Bodies")
    plt.figtext(0.025, 0.92, f"Total Area: {yearly_hardiness_area.loc[yearly_hardiness_area.index.year == year].values[0]:.2f} km²", fontsize=12, bbox={"facecolor": "white", "alpha": 0.5, "pad": 5})
    
    # Plot contour for elevation
    #yearly_temp_df.plot(column="elevation", ax=ax, markersize=3, cmap="BuPu", alpha=0.9, legend=True, legend_kwds={'label': "Elevation (m)", 'shrink': 0.5})

    # Plot total precipitation
    #yearly_temp_df.plot(column="tp", ax=ax, markersize=3, cmap=customGreen, alpha=0.9, legend=True, legend_kwds={'label': "Total Precipitation (mm)", 'shrink': 0.5})

    # Plot boundary
    gpd.GeoSeries(boundary).to_crs(epsg=3857).plot(ax=ax, facecolor=("red", 0.125), label="Boundaries", edgecolor=("black", 0.8), linewidth=1.5)

    # Add basemap
    ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

    ax.set_title(f"Hardiness Zones - {year}", fontsize=15)
    plt.axis("off")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"./results/{coffee_type}/hardiness_zones_{year}.jpeg")


In [28]:
# Convert data for HeatMapWithTime
time_index = temp_df["time"].sort_values().astype("str").unique().tolist()
#geo_df.sort_values(by="time", inplace=True)
# Group by time, convert to [[lon, lat, value], ...]
hardiness_zones = [
    d[["geometry", "t2m"]]
    .assign(lon=lambda df: df.geometry.y, lat=lambda df: df.geometry.x)
    .loc[:, ["lon", "lat", "t2m"]]
    .values.tolist()
    for _, d in temp_df.groupby("time")
]

# total_precipitation = [
#     d[["geometry", "tp"]]
#     .assign(lon=lambda df: df.geometry.y, lat=lambda df: df.geometry.x)
#     .loc[:, ["lon", "lat", "tp"]]
#     .values.tolist()
#     for _, d in temp_df.groupby("time")
# ]

# elevation = [
#     d[["geometry", "elevation"]]
#     .assign(lon=lambda df: df.geometry.y, lat=lambda df: df.geometry.x)
#     .loc[:, ["lon", "lat", "elevation"]]
#     .values.tolist()
#     for _, d in temp_df.groupby("time")
# ]


map = folium.Map(location=[9.14, 40.48], tiles="OpenStreetMap", zoom_start=5, control_scale=True) # OpenStreeMap. 'cartodbpositron', stamentoner
HeatMapWithTime(
    hardiness_zones,
    name="Hardiness Zones",
    index=time_index,
    use_local_extrema=True, 
    max_opacity=0.5,
    radius=10,
    
).add_to(map)

# HeatMapWithTime(
#     total_precipitation,
#     name="Total Precipitation",
#     index=time_index,
#     use_local_extrema=True, 
#     max_opacity=0.5,
#     radius=10,
#     show=False
    
# ).add_to(map)

# HeatMapWithTime(
#     elevation,
#     name="Elevation",
#     index=time_index,
#     use_local_extrema=True, 
#     max_opacity=0.5,
#     radius=10,
#     show=False
# ).add_to(map)


folium.LayerControl().add_to(map)

map.save("map.html")

In [104]:
# Cluster data
year = 2025
yearly_temp_df = temp_df[temp_df["time"].dt.year == year]
yearly_clustered = clustered_hardiness_zones[clustered_hardiness_zones["time"].dt.year == year]


map = folium.Map(location=[8, 40], 
                 tiles="OpenStreetMap", 
                 zoom_start=5,
                 min_zoom=4,
                 #control_scale=True,
                 ) 

# Add red boundary
boundary_geojson = gpd.GeoDataFrame({'geometry': [boundary_union]}, crs="EPSG:4326")
boundary_json = boundary_geojson.to_json()
folium.GeoJson(
    boundary_json,
    name=f'{countries}',
    style_function=lambda x: {
        'fillColor': 'red',  # Keep red fill
        'color': 'red',
        'weight': 2,
        'opacity': 0.1,  # Lower opacity so heatmap shows through
        'fillOpacity': 0.1
    }
).add_to(map)

# Hardiness zones
folium.GeoJson(
    yearly_clustered[["geometry"]], 
    name="Hardiness Zones", 
    marker=folium.Circle(radius=0.5, color="green"),
    style_function=lambda x: {
        "color": "green",
        'opacity': 0.5,  # Lower opacity so heatmap shows through
    }
).add_to(map)


# # hardiness points
# folium.GeoJson(
#     yearly_temp_df[["geometry"]], 
#     name="Hardiness Zones", 
#     marker=folium.Circle(radius=0.1, color="green"), 
#     style_function=lambda x: {
#         'opacity': 0.3,  # Lower opacity so heatmap shows through
#     }

# ).add_to(map)


# # Timeline hardiness zones
# styledict = {}

# on = {
#     "color": "green",
#     "opacity": 0.4,
# } 

# off = {
#     "color": "blue",
#     "opacity": 0,
# }

# clustered_hardiness_zones["elapse_time"] = clustered_hardiness_zones.time.astype("int64") // 10 ** 9  
# final_year = clustered_hardiness_zones["elapse_time"].max()

# for idx, data in clustered_hardiness_zones.iterrows(): 
#     styledict[str(idx)] = {
#         #data.elapse_time - 31536000: off,
#         data.elapse_time-1: off,
#         data.elapse_time: on,
#         data.elapse_time+1: off,
#     }

# TimeSliderChoropleth(
#     clustered_hardiness_zones["geometry"].to_json(),
#     date_options="YYYY",
#     styledict=styledict,
#     stroke_opacity=0,
#     overlay=True,
# ).add_to(map)


# Add title 
map_title = f"Hardiness Zones Map - {year}"
title_html = f'<h1 style="position:absolute;z-index:100000;left:40vw" >{map_title}</h1>'
map.get_root().html.add_child(folium.Element(title_html))

map.save("map.html")


### Load and Cluster Farmlands

In [3]:
# query = """
# area["ISO3166-1"="KE"]->.ke;
# area["ISO3166-1"="ET"]->.et;
# area["ISO3166-1"="UG"]->.ug;

# (
#   way(area.ke)[landuse~"plantation|forest|farmland"];
#   way(area.et)[landuse~"plantation|forest|farmland"];
#   way(area.ug)[landuse~"plantation|forest|farmland"];
# );
# out geom;
# """

query = """
[out:json][timeout:1800];

// Define country relations
rel["ISO3166-1"="KE"]->.rKE;
rel["ISO3166-1"="ET"]->.rET;
rel["ISO3166-1"="UG"]->.rUG;

(.rKE; .rET; .rUG;)->.rels;
.rels map_to_area -> .countries;

// Query farmland
(
  way["landuse"="farmland"](area.countries);
  relation["landuse"="farmland"](area.countries);
);
out body;
>;
out skel qt;
"""


api = overpy.Overpass()
# Ask the client to return GeoJSON directly
res = api.query(query)
print(res)


# # Save to GeoJSON
# out_path = "./dataset/landuse_ke_et_ug.geojson"
# with open(out_path, "w", encoding="utf-8") as f:
#     json.dump(res, f, ensure_ascii=False)

# print(f"Saved: {out_path} | Features: {len(res.get('features', []))}")

IncompleteRead: IncompleteRead(3193 bytes read)

In [20]:
kenya_farmlands = gpd.read_file("./dataset/farmland/kenya.geojson")
uganda_farmlands = gpd.read_file("./dataset/farmland/uganda.geojson")
ethiopia_farmlands = gpd.read_file("./dataset/farmland/ethiopia.geojson")

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
boundaries[boundaries["ADMIN"] == "Kenya"].plot(ax=ax, alpha=0.5)
# gpd.GeoSeries(boundary).plot(ax=ax, cmap="tab20b", alpha=0.5)
farmlands.plot(ax=ax, color="green", alpha=1)

In [22]:
map = folium.Map(location=[8, 40], 
                 tiles="OpenStreetMap", 
                 zoom_start=5,
                 min_zoom=4,
                 #control_scale=True,
                 ) 

folium.GeoJson(
    kenya_farmlands[["geometry"]], 
    name="Kenya Farmlands", 
    marker=folium.Circle(radius=0.5, color="green"),
    style_function=lambda x: {
        "color": "green",
        'opacity': 1,  # Lower opacity so heatmap shows through
    }
).add_to(map)


folium.GeoJson(
    uganda_farmlands[["geometry"]], 
    name="Uganda Farmlands", 
    marker=folium.Circle(radius=0.5, color="green"),
    style_function=lambda x: {
        "color": "green",
        'opacity': 1,  # Lower opacity so heatmap shows through
    }
).add_to(map)


folium.GeoJson(
    ethiopia_farmlands[["geometry"]], 
    name="Ethiopia Farmlands", 
    marker=folium.Circle(radius=0.5, color="green"),
    style_function=lambda x: {
        "color": "green",
        'opacity': 1,  # Lower opacity so heatmap shows through
    }
).add_to(map)

map.save("farmlands_map.html")
